# CS224N Lecture 3

In [ ]:
import random
import numpy as np
from utils.treebank import StanfordSentiment
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import time

from word2vec import *
from sgd import *

# Check Python Version
import sys
assert sys.version_info[0] == 3
assert sys.version_info[1] >= 5

C:\Users\AILAB\AppData\Local\Temp\ipykernel_46696\3245635959.py:5: MatplotlibDeprecationWarning: Auto-close()ing of figures upon backend switching is deprecated since 3.8 and will be removed in 3.10.  To suppress this warning, explicitly call plt.close('all') first.
  matplotlib.use('TkAgg')


In [2]:
# Reset the random seed to make sure that everyone gets the same results
random.seed(314)
dataset = StanfordSentiment()
tokens = dataset.tokens()
nWords = len(tokens)

# We are going to train 10-dimensional vectors for this assignment
dimVectors = 10

# Context size
C = 5

In [3]:
# Reset the random seed to make sure that everyone gets the same results
random.seed(31415)
np.random.seed(9265)

In [4]:
startTime=time.time()
wordVectors = np.concatenate(
    ((np.random.rand(nWords, dimVectors) - 0.5) /
       dimVectors, np.zeros((nWords, dimVectors))),
    axis=0)
wordVectors = sgd(
    lambda vec: word2vec_sgd_wrapper(skipgram, tokens, vec, dataset, C,
        negSamplingLossAndGradient),
    wordVectors, 0.3, 40000, None, True, PRINT_EVERY=10)
# Note that normalization is not called here. This is not a bug,
# normalizing during training loses the notion of length.

iter 10010: 19.061543
iter 10020: 19.320780
iter 10030: 19.391688
iter 10040: 19.581040
iter 10050: 19.638938
iter 10060: 19.732060
iter 10070: 19.927272
iter 10080: 19.845862
iter 10090: 20.126880
iter 10100: 20.302353
iter 10110: 20.377556
iter 10120: 20.288877
iter 10130: 20.334256
iter 10140: 20.392606
iter 10150: 20.432798
iter 10160: 20.486230
iter 10170: 20.376875
iter 10180: 20.555091
iter 10190: 20.556658
iter 10200: 20.870761
iter 10210: 21.054778
iter 10220: 21.084734
iter 10230: 21.235188
iter 10240: 21.225626
iter 10250: 21.308033
iter 10260: 21.332949
iter 10270: 21.387118
iter 10280: 21.438577
iter 10290: 21.464586
iter 10300: 21.344436
iter 10310: 21.405661
iter 10320: 21.433333
iter 10330: 21.459616
iter 10340: 21.454081
iter 10350: 21.662324
iter 10360: 21.677140
iter 10370: 21.706479
iter 10380: 21.543729
iter 10390: 21.678858
iter 10400: 21.639473
iter 10410: 21.678314
iter 10420: 21.783850
iter 10430: 21.845981
iter 10440: 21.752499
iter 10450: 21.747567
iter 10460

In [5]:
print("sanity check: cost at convergence should be around or below 10")
print("training took %d seconds" % (time.time() - startTime))

sanity check: cost at convergence should be around or below 10
training took 3713 seconds


In [6]:
# concatenate the input and output word vectors
wordVectors = np.concatenate(
    (wordVectors[:nWords,:], wordVectors[nWords:,:]),
    axis=0)

In [9]:
visualizeWords = [
    "great", "cool", "brilliant", "wonderful", "well", "amazing",
    "worth", "sweet", "enjoyable", "boring", "bad", "dumb",
    "annoying", "female", "male", "queen", "king", "man", "woman", "rain", "snow",
    "hail", "coffee", "tea"]

visualizeIdx = [tokens[word] for word in visualizeWords]
visualizeVecs = wordVectors[visualizeIdx, :]
temp = (visualizeVecs - np.mean(visualizeVecs, axis=0))
covariance = 1.0 / len(visualizeIdx) * temp.T.dot(temp)
U,S,V = np.linalg.svd(covariance)
coord = temp.dot(U[:,0:2])

for i in range(len(visualizeWords)):
    plt.text(coord[i,0], coord[i,1], visualizeWords[i],
        bbox=dict(facecolor='green', alpha=0.1))

plt.xlim((np.min(coord[:,0]), np.max(coord[:,0])))
plt.ylim((np.min(coord[:,1]), np.max(coord[:,1])))
plt.savefig('word_vectors.png')